<a href="https://colab.research.google.com/github/databyhuseyn/DeepLearning/blob/main/Titanic_SK_Learn_vs_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

In [ ]:
if train['Survived'].isna().sum() > 0:
  train.dropna(subset=['Survived'])

In [ ]:
train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [ ]:
X_train_full = train.drop(['Survived'], axis=1)
y_train_full = train['Survived']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.2, stratify=y_train_full)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

In [ ]:
cat_features = X_train.select_dtypes(exclude=[np.number]).columns
num_features = X_train.select_dtypes(include=[np.number]).columns

In [ ]:
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='Missing')),
    ('onehot', OneHotEncoder(handle_unknown ='ignore', sparse_output=False))
])

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

transformer = ColumnTransformer([
    ('cat', cat_pipeline, cat_features),
    ('num', num_pipeline, num_features)
], remainder='passthrough')

full_pipeline = Pipeline([
    ('preprocessing', transformer),
    ('rfc', RandomForestClassifier())
])

In [ ]:
full_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='Missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  Index(['Sex', 'Embarked'], dtype='object')),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object'))])),
                ('rfc', RandomForestClassifier())])

In [ ]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
552,3,male,NaN,0,0,7.8292,Q
72,2,male,21.0,0,0,73.5000,S
537,1,female,30.0,0,0,106.4250,C
571,1,female,53.0,2,0,51.4792,S
73,3,male,26.0,1,0,14.4542,C
...,...,...,...,...,...,...,...
378,3,male,20.0,0,0,4.0125,C
173,3,male,21.0,0,0,7.9250,S
546,2,female,19.0,1,0,26.0000,S
282,3,male,16.0,0,0,9.5000,S


In [ ]:
X_valid

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
130,3,male,33.0,0,0,7.8958,C
258,1,female,35.0,0,0,512.3292,C
223,3,male,NaN,0,0,7.8958,S
529,2,male,23.0,2,1,11.5000,S
611,3,male,NaN,0,0,7.0500,S
...,...,...,...,...,...,...,...
721,3,male,17.0,1,0,7.0542,S
583,1,male,36.0,0,0,40.1250,C
317,2,male,54.0,0,0,14.0000,S
244,3,male,30.0,0,0,7.2250,C


In [ ]:
full_pipeline.score(X_train, y_train)

0.9859550561797753

In [ ]:
full_pipeline.score(X_valid, y_valid)

0.8156424581005587

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    'rfc__n_estimators': [500, 700, 900],
    'rfc__criterion': ['gini', 'entropy', 'log_loss'],
    'rfc__n_jobs' : [-1],
    'rfc__max_features': ['sqrt', 'log2']
}

grid_cv = GridSearchCV(full_pipeline, param_grid, cv = 5)
grid_cv.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(fill_value='Missing',
                                                                                                        strategy='constant')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse_output=False))]),
                                                                         Index(['Sex', 'Embarked'], dtype='object')),
                                                                        ('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object'))])),
                                       ('rfc', RandomForestClassifier())]),
             param_grid={'rfc__criterion': ['gini', 'entropy', 'log_loss'],
                         'rfc__max_features': ['sqrt', 'log2'],
                         'rfc__n_estimators': [500, 700, 900],
                         'rfc__n_jobs': [-1]})

In [ ]:
grid_cv.best_params_

{'rfc__criterion': 'log_loss',
 'rfc__max_features': 'log2',
 'rfc__n_estimators': 500,
 'rfc__n_jobs': -1}

In [ ]:
grid_cv.best_estimator_.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='Missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  Index(['Sex', 'Embarked'], dtype='object')),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object'))])),
                ('rfc',
                 RandomForestClassifier(criterion='log_loss',
                                        max_features='log2', n_estimators=500,
                                        n_jobs=-1))])

In [ ]:
grid_cv.best_estimator_.score(X_train, y_train) , grid_cv.best_estimator_.score(X_valid, y_valid)

(0.9859550561797753, 0.8156424581005587)

In [ ]:
import tensorflow as tf

In [ ]:
X_train.shape[1:][0]

7

In [ ]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=[11]),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# model.fit(tf.constant(X_train), tf.constant(y_train))

In [ ]:
from tensorflow.keras.metrics import F1Score

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)


model.compile(loss='BinaryCrossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
X_train_transformed = transformer.fit_transform(X_train)

In [ ]:
X_train_transformed_df = pd.DataFrame(X_train_transformed, columns = transformer.get_feature_names_out())

In [ ]:
X_train_transformed_df

,cat__Sex_female,cat__Sex_male,cat__Embarked_C,cat__Embarked_Missing,cat__Embarked_Q,cat__Embarked_S,num__Pclass,num__Age,num__SibSp,num__Parch,num__Fare
0,0.0,1.0,0.0,0.0,1.0,0.0,0.840740,-0.103876,-0.460719,-0.459792,-0.487963
1,0.0,1.0,0.0,0.0,0.0,1.0,-0.368567,-0.649988,-0.460719,-0.459792,0.892853
2,1.0,0.0,1.0,0.0,0.0,0.0,-1.577873,0.052156,-0.460719,-0.459792,1.585146
3,1.0,0.0,0.0,0.0,0.0,1.0,-1.577873,1.846522,1.419121,-0.459792,0.429837
4,0.0,1.0,1.0,0.0,0.0,0.0,0.840740,-0.259908,0.479201,-0.459792,-0.348664
...,...,...,...,...,...,...,...,...,...,...,...
707,0.0,1.0,1.0,0.0,0.0,0.0,0.840740,-0.728004,-0.460719,-0.459792,-0.568214
708,0.0,1.0,0.0,0.0,0.0,1.0,0.840740,-0.649988,-0.460719,-0.459792,-0.485949
709,1.0,0.0,0.0,0.0,0.0,1.0,-0.368567,-0.806019,0.479201,-0.459792,-0.105898
710,0.0,1.0,0.0,0.0,0.0,1.0,0.840740,-1.040067,-0.460719,-0.459792,-0.452832


In [ ]:
model.compile(loss='BinaryCrossentropy', optimizer=optimizer, metrics=['accuracy'])
model.fit(X_train_transformed_df, y_train, epochs=200)

Epoch 1/200
23/23 [==============================] - 3s 7ms/step - loss: 0.5509 - accuracy: 0.7261
Epoch 2/200
23/23 [==============================] - 0s 3ms/step - loss: 0.4444 - accuracy: 0.7992
Epoch 3/200
23/23 [==============================] - 0s 4ms/step - loss: 0.4206 - accuracy: 0.8272
Epoch 4/200
23/23 [==============================] - 0s 3ms/step - loss: 0.4132 - accuracy: 0.8301
Epoch 5/200
23/23 [==============================] - 0s 3ms/step - loss: 0.4062 - accuracy: 0.8301
Epoch 6/200
23/23 [==============================] - 0s 4ms/step - loss: 0.4051 - accuracy: 0.8385
Epoch 7/200
23/23 [==============================] - 0s 4ms/step - loss: 0.4008 - accuracy: 0.8216
Epoch 8/200
23/23 [==============================] - 0s 3ms/step - loss: 0.3929 - accuracy: 0.8385
Epoch 9/200
23/23 [==============================] - 0s 4ms/step - loss: 0.3921 - accuracy: 0.8413
Epoch 10/200
23/23 [==============================] - 0s 4ms/step - loss: 0.3875 - accuracy: 0.8441
Epoch 11/

In [ ]:
inputs = tf.keras.Input(shape=[11])
hidden1 = tf.keras.layers.Dense(1024, activation='relu')(inputs)
hidden2 = tf.keras.layers.Dense(512, activation='relu')(hidden1)
hidden3 = tf.keras.layers.Dense(256, activation='relu')(hidden2)
main_output = tf.keras.layers.Dense(1, activation='sigmoid')(hidden3)

In [ ]:
model = tf.keras.Model(inputs, main_output)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 11)]              0         
                                                                 
 dense_3 (Dense)             (None, 1024)              12288     
                                                                 
 dense_4 (Dense)             (None, 512)               524800    
                                                                 
 dense_5 (Dense)             (None, 256)               131328    
                                                                 
 dense_6 (Dense)             (None, 1)                 257       
                                                                 
Total params: 668673 (2.55 MB)
Trainable params: 668673 (2.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss='BinaryCrossentropy', optimizer=optimizer, metrics='Accuracy')

In [ ]:
model.fit(X_train_transformed_df, y_train, epochs=200)

Epoch 1/200
23/23 [==============================] - 2s 4ms/step - loss: 0.5043 - Accuracy: 0.7598
Epoch 2/200
23/23 [==============================] - 0s 4ms/step - loss: 0.4258 - Accuracy: 0.8258
Epoch 3/200
23/23 [==============================] - 0s 4ms/step - loss: 0.4326 - Accuracy: 0.8230
Epoch 4/200
23/23 [==============================] - 0s 4ms/step - loss: 0.4138 - Accuracy: 0.8385
Epoch 5/200
23/23 [==============================] - 0s 4ms/step - loss: 0.4041 - Accuracy: 0.8272
Epoch 6/200
23/23 [==============================] - 0s 4ms/step - loss: 0.3981 - Accuracy: 0.8315
Epoch 7/200
23/23 [==============================] - 0s 4ms/step - loss: 0.3842 - Accuracy: 0.8511
Epoch 8/200
23/23 [==============================] - 0s 4ms/step - loss: 0.3890 - Accuracy: 0.8469
Epoch 9/200
23/23 [==============================] - 0s 4ms/step - loss: 0.3873 - Accuracy: 0.8371
Epoch 10/200
23/23 [==============================] - 0s 4ms/step - loss: 0.3722 - Accuracy: 0.8455
Epoch 11/

In [ ]:
X_valid_transformed = transformer.transform(X_valid)
X_valid_transformed_df = pd.DataFrame(X_valid_transformed, columns = transformer.get_feature_names_out())
X_valid_transformed_df

,cat__Sex_female,cat__Sex_male,cat__Embarked_C,cat__Embarked_Missing,cat__Embarked_Q,cat__Embarked_S,num__Pclass,num__Age,num__SibSp,num__Parch,num__Fare
0,0.0,1.0,1.0,0.0,0.0,0.0,0.840740,0.286203,-0.460719,-0.459792,-0.486563
1,1.0,0.0,1.0,0.0,0.0,0.0,-1.577873,0.442235,-0.460719,-0.459792,10.119825
2,0.0,1.0,0.0,0.0,0.0,1.0,0.840740,-0.103876,-0.460719,-0.459792,-0.486563
3,0.0,1.0,0.0,0.0,0.0,1.0,-0.368567,-0.493956,1.419121,0.865601,-0.410780
4,0.0,1.0,0.0,0.0,0.0,1.0,0.840740,-0.103876,-0.460719,-0.459792,-0.504347
...,...,...,...,...,...,...,...,...,...,...,...
174,0.0,1.0,0.0,0.0,0.0,1.0,0.840740,-0.962051,0.479201,-0.459792,-0.504258
175,0.0,1.0,1.0,0.0,0.0,0.0,-1.577873,0.520251,-0.460719,-0.459792,0.191099
176,0.0,1.0,0.0,0.0,0.0,1.0,-0.368567,1.924538,-0.460719,-0.459792,-0.358214
177,0.0,1.0,1.0,0.0,0.0,0.0,0.840740,0.052156,-0.460719,-0.459792,-0.500667


In [ ]:
X_valid

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
130,3,male,33.0,0,0,7.8958,C
258,1,female,35.0,0,0,512.3292,C
223,3,male,NaN,0,0,7.8958,S
529,2,male,23.0,2,1,11.5000,S
611,3,male,NaN,0,0,7.0500,S
...,...,...,...,...,...,...,...
721,3,male,17.0,1,0,7.0542,S
583,1,male,36.0,0,0,40.1250,C
317,2,male,54.0,0,0,14.0000,S
244,3,male,30.0,0,0,7.2250,C


In [ ]:
y_valid

130    0
258    1
223    0
529    0
611    0
      ..
721    0
583    0
317    0
244    0
554    1
Name: Survived, Length: 179, dtype: int64

In [ ]:
import shutil

shutil.rmtree("my_keras_model", ignore_errors=True)

In [ ]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('My checkpoints',
                                   save_weights_only=True)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10,
                                                     restore_best_weights=True)

In [ ]:
history = model.fit(X_train_transformed_df, y_train, epochs=1000,
          validation_data=(X_valid_transformed_df, y_valid), callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/1000
23/23 [==============================] - 0s 17ms/step - loss: 0.1823 - Accuracy: 0.9171 - val_loss: 1.2708 - val_Accuracy: 0.7877
Epoch 2/1000
23/23 [==============================] - 0s 8ms/step - loss: 0.1789 - Accuracy: 0.9213 - val_loss: 1.3008 - val_Accuracy: 0.7709
Epoch 3/1000
23/23 [==============================] - 0s 7ms/step - loss: 0.1792 - Accuracy: 0.9199 - val_loss: 1.3070 - val_Accuracy: 0.7821
Epoch 4/1000
23/23 [==============================] - 0s 7ms/step - loss: 0.1834 - Accuracy: 0.9185 - val_loss: 1.3408 - val_Accuracy: 0.7821
Epoch 5/1000
23/23 [==============================] - 0s 7ms/step - loss: 0.1794 - Accuracy: 0.9143 - val_loss: 1.3509 - val_Accuracy: 0.7877
Epoch 6/1000
23/23 [==============================] - 0s 12ms/step - loss: 0.1841 - Accuracy: 0.9171 - val_loss: 1.3777 - val_Accuracy: 0.7933
Epoch 7/1000
23/23 [==============================] - 0s 12ms/step - loss: 0.1943 - Accuracy: 0.9171 - val_loss: 1.3368 - val_Accuracy: 0.8045
Epo

In [ ]:
class shallowTitanic(tf.keras.Model):
  def __init__(self, unit=30, activation='relu', **kwargs):
    super().__init__(**kwargs)
    self.inputs = tf.keras.Input(shape=[11])
    self.hidden1 = tf.keras.layers.Dense(unit, activation='relu')
    self.hidden2 = tf.keras.layers.Dense(unit, activation='relu')
    self.hidden3 = tf.keras.layers.Dense(unit, activation='relu')
    self.outputs = tf.keras.layers.Dense(1, activation='sigmoid')

  def call(self, inputs):
    layer1 = self.hidden1(inputs)
    layer2 = self.hidden2(layer1)
    layer3 = self.hidden3(layer2)
    output = self.outputs(layer3)
    return output

tf.random.set_seed(42)
model = shallowTitanic()

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss='BinaryCrossentropy', optimizer=optimizer, metrics=['accuracy'])
model.fit(X_train_transformed_df, y_train, epochs=1000,
          validation_data=(X_valid_transformed_df, y_valid),
          callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/1000
23/23 [==============================] - 2s 15ms/step - loss: 0.6458 - accuracy: 0.6433 - val_loss: 0.5903 - val_accuracy: 0.6872
Epoch 2/1000
23/23 [==============================] - 0s 7ms/step - loss: 0.5705 - accuracy: 0.7107 - val_loss: 0.5211 - val_accuracy: 0.7039
Epoch 3/1000
23/23 [==============================] - 0s 7ms/step - loss: 0.5077 - accuracy: 0.7837 - val_loss: 0.4875 - val_accuracy: 0.7542
Epoch 4/1000
23/23 [==============================] - 0s 7ms/step - loss: 0.4672 - accuracy: 0.8048 - val_loss: 0.4624 - val_accuracy: 0.7877
Epoch 5/1000
23/23 [==============================] - 0s 6ms/step - loss: 0.4453 - accuracy: 0.8034 - val_loss: 0.4472 - val_accuracy: 0.8045
Epoch 6/1000
23/23 [==============================] - 0s 7ms/step - loss: 0.4330 - accuracy: 0.8118 - val_loss: 0.4402 - val_accuracy: 0.7877
Epoch 7/1000
23/23 [==============================] - 0s 7ms/step - loss: 0.4222 - accuracy: 0.8118 - val_loss: 0.4374 - val_accuracy: 0.8045
Epoch